In [1]:
import cv2
import numpy as np
from scipy.signal import medfilt

In [ ]:
def mean_color_diff(frame1, frame2):
  mean1 = np.mean(frame1, axis=(0, 1))
  mean2 = np.mean(frame2, axis=(0, 1))

  return np.linalg.norm(mean1 - mean2)

    def mean_color_diff_hsv(frame1, frame2):
    hsv1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)
    mean1 = np.mean(hsv1, axis=(0, 1))
    mean2 = np.mean(hsv2, axis=(0, 1))
    
    # Нормализуем Hue (0–179 в OpenCV), чтобы избежать скачков 0↔179
    h_diff = min(abs(mean1[0] - mean2[0]), 180 - abs(mean1[0] - mean2[0]))
    s_diff = abs(mean1[1] - mean2[1])
    v_diff = abs(mean1[2] - mean2[2])
    
    return np.sqrt(h_diff**2 + s_diff**2 + v_diff**2)

In [2]:
def luminance_diff(frame1, frame2, resize=(64,64)):
  y1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2YUV)[:, :, 0].astype(np.float32) / 255.0
  y2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2YUV)[:, :, 0].astype(np.float32) / 255.0
  y1 = cv2.resize(y1, resize)
  y2 = cv2.resize(y2, resize)
  return abs(np.mean(y1) - np.mean(y2))

In [3]:
def luminance_median_diff(frame1, frame2):
  y1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2YUV)[:, :, 0].astype(np.float32) / 255.0
  y2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2YUV)[:, :, 0].astype(np.float32) / 255.0

  y1_med = np.median(y1)
  y2_med = np.median(y2)
  return abs(y1_med - y2_med)

In [4]:
def luminance_diff_avg(frame1, frame2):
  y1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2YUV)[:, :, 0]
  y2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2YUV)[:, :, 0]
  y1_avg = np.average(y1 / 255.0)
  y2_avg = np.average(y2 / 255.0)
  return abs(y1_avg - y2_avg)

In [5]:
def block_luminance_diff(frame1, frame2, grid=(4,4)):
  y1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2YUV)[:, :, 0].astype(np.float32) / 255.0
  y2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2YUV)[:, :, 0].astype(np.float32) / 255.0

  h, w = y1.shape
  bh, bw = h // grid[0], w // grid[1]

  diff = 0.0
  for i in range(grid[0]):
    for j in range(grid[1]):
      y1_block = y1[i*bh:(i+1)*bh, j*bw:(j+1)*bw]
      y2_block = y2[i*bh:(i+1)*bh, j*bw:(j+1)*bw]
      diff += abs(np.mean(y1_block) - np.mean(y2_block))
  return diff

In [6]:
def luminance_stats_diff(frame1, frame2):
  y1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2YUV)[:, :, 0].astype(np.float32) / 255.0
  y2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2YUV)[:, :, 0].astype(np.float32) / 255.0

  mu1, sigma1 = np.mean(y1), np.std(y1)
  mu2, sigma2 = np.mean(y2), np.std(y2)

  return abs(mu1 - mu2) + abs(sigma1 - sigma2)

In [7]:
def luminance_and_edge_diff(frame1, frame2, alpha=0.7):
  y1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2YUV)[:,:,0].astype(np.float32)
  y2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2YUV)[:,:,0].astype(np.float32)
  lum_diff = np.mean(np.abs(y1 - y2)) / 255.0

  edges1 = cv2.Sobel(y1, cv2.CV_64F, 1, 1, ksize=3)
  edges2 = cv2.Sobel(y2, cv2.CV_64F, 1, 1, ksize=3)
  edge_diff = np.mean(np.abs(edges1 - edges2))

  return alpha * lum_diff + (1 - alpha) * edge_diff

In [23]:
def adaptive_threshold_mad(diff_history, current_diff, window_size=50, threshold_factor=3.0):
  if len(diff_history) < window_size:
    return False

  recent_diffs = diff_history[-window_size:]
  median = np.median(recent_diffs)
  mad = np.median(abs(recent_diffs - median))

  if mad == 0:
    return False

  mad_score = (current_diff - median) / (1.4826 * mad)
  return mad_score > threshold_factor

In [24]:
def isLocal_peak(diffs, frameidx, peak_window=3):
  start = max(0, frameidx - peak_window)
  end = min(len(diffs), frameidx + peak_window + 1)
  local_max = np.max(diffs[start:end])
  return diffs[frameidx] == local_max

In [28]:
def detect_scene(video_path, threshold_factor=3.0, window_size=3):
  cap = cv2.VideoCapture(video_path)
  diffs = []
  prive_frame = None
  scene_change = []

  while cap.isOpened():
    ref, frame = cap.read()
    if not ref:
      break
    if prive_frame is not None:
      d = luminance_diff(prive_frame, frame)
      diffs.append(d)

      if adaptive_threshold_mad(diffs, d, window_size=window_size, threshold_factor=threshold_factor):
        current_frameidx = len(diffs) - 1
        if isLocal_peak(diffs, current_frameidx, peak_window=window_size):
          scene_change.append(current_frameidx + 1)
    prive_frame = frame.copy()
  cap.release()
  return scene_change, np.array(diffs)

In [29]:
def seconds(frame, video_path):
  cap = cv2.VideoCapture(video_path)
  fps = cap.get(cv2.CAP_PROP_FPS)
  second = frame / fps
  return second, fps

In [40]:
scene_change, diffs = detect_scene('/content/drive/MyDrive/practical_data/man_in_costume.mp4', threshold_factor=5.0, window_size=10)
print('Кадры смены сцен', scene_change)

Кадры смены сцен [47, 103, 139, 140, 189, 205, 246, 263, 297, 363, 368, 390, 391]


In [41]:
len(scene_change)

13

In [ ]:
for frame in scene_change:
  second, fps = seconds(frame,'/content/drive/MyDrive/practical_data/man_in_costume.mp4')
  print(f'Кадр под номером {frame} был на {second:.2f} секунду')
print(f'FPS in video -> {fps}')

Кадр под номером 47 был на 1.57 секунду
Кадр под номером 103 был на 3.43 секунду
Кадр под номером 139 был на 4.63 секунду
Кадр под номером 140 был на 4.67 секунду
Кадр под номером 142 был на 4.73 секунду
Кадр под номером 144 был на 4.80 секунду
Кадр под номером 168 был на 5.60 секунду
Кадр под номером 177 был на 5.90 секунду
Кадр под номером 189 был на 6.30 секунду
Кадр под номером 213 был на 7.10 секунду
Кадр под номером 246 был на 8.20 секунду
Кадр под номером 297 был на 9.90 секунду
Кадр под номером 337 был на 11.23 секунду
Кадр под номером 368 был на 12.27 секунду
Кадр под номером 390 был на 13.00 секунду
Кадр под номером 391 был на 13.03 секунду
FPS in video -> 30.0


In [ ]:
scene_change, diffs = detect_scene('/content/drive/MyDrive/practical_data/man_presentation.mp4', threshold=2.0)
print('Кадры под номером', scene_change)

Кадры под номером [51, 97, 141, 194, 265, 333, 395, 444, 512, 576, 660, 669, 670, 778, 779, 792, 875, 876, 877, 887, 888, 891, 892, 899, 900, 901, 902, 907, 908, 909, 910, 911, 912, 923, 924, 926, 927, 928, 932, 933, 935, 936, 937, 938, 940, 941, 946, 950, 956, 1001, 1053, 1110, 1175, 1223, 1540, 1582, 1590, 1597, 1598, 1605, 1606, 1607, 1622, 1684, 1714, 1741, 1789, 1805, 1860, 1885, 1930, 2120, 2123, 2124, 2125, 2126, 2178, 2246, 2349, 2415, 2416, 2417, 2420, 2735, 2795, 2796, 2797, 2798, 2799, 2800, 2801, 2802, 2803, 2804, 2805, 2806, 2901, 3015, 3019, 3025, 3057, 3058, 3071, 3163, 3232, 3283, 3285, 3286, 3287, 3288, 3290, 3291]


In [ ]:
len(scene_change)

112

16 - luminance_diff with YUV (man_in_costume.mp4)

work_time: 4s

10 - grid_luminance_diff (man_in_costume.mp4)

work_time: 5s

19 - luminance_stats_diff (man_in_costume.mp4)

work_time: 5s

1 - luminance_and_edge_diff (man_in_costume.mp4)

work_time: 14s

17 - luminance_median_diff (man_in_costume.mp4)

work_time: 8s

16 - default_luminance + resize (man_in_costume.mp4)

work_time: 5s

-------------------------------------------------------------------------------

125 - luminance_diff with YUV (man_presentation.mp4)

work_time: 1.20min

130 - grid_luminance_diff (man_presentation.mp4)

work_time1.30min

127 - luminance_stats_diff (man_presentation.mp4)

work_time: 1.50min

26 - luminance_and_edge_diff (msn_presentation.mp4)

work_time: 3min

150 - luminance_median_diff (man_presentation.mp4)

work_time: 2min

112 - defalt_luminance + resize (man_presentation.mp4)

work_time ~1min